# Pytorch加载数据
dataset -> dataloader

In [8]:
from torch.utils.data import Dataset
from PIL import Image
import os

class my_data(Dataset):

    def __init__(self, root_dir, label_dir) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.img_path = os.listdir(self.path)
    
    def __getitem__(self, index):
        img_name = self.img_path[index]
        img_item_path = os.path.join(self.root_dir, self.label_dir, img_name)
        img = Image.open(img_item_path)
        label = self.label_dir
        return img, label
    
    def __len__(self):
        return len(self.img_path)

root_dir = '/Users/chen/Desktop/练手数据集/train'
ants_image_dir = '/Users/chen/Desktop/练手数据集/train/ants_image'
ants_dataset = my_data(root_dir, ants_image_dir)

img, label = ants_dataset[0]
len(ants_dataset)

    


124